In [1]:
import pandas as pd

In [2]:
import os
path_of_the_directory = '.'
ext = ('.csv')
All_dfs = {}
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext):
        print(file) 
        temp = (file.replace("_"," ").replace("."," ").split(" ")[0].lower())
        All_dfs[temp] = pd.read_csv(file)
    else:
        continue

In [3]:
SRIntShip = All_dfs["internalshipments"].query('from_to_where == "SR"')
SRIntShip = SRIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SRIntShip

KeyError: 'internalshipments'

In [ ]:
RCExtShip = All_dfs["externalshipments"].query('from_to_where == "RC"')
RCExtShip = RCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#RCExtShip

In [ ]:
SSIntShip = All_dfs["internalshipments"].query('from_to_where == "SS"')
SSIntShip = SSIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SSIntShip

In [ ]:
SCExtShip = All_dfs["externalshipments"].query('from_to_where == "SC"')
SCExtShip = SCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SCExtShip

In [ ]:
All_dfs["RCExtShip".lower()] = RCExtShip
All_dfs["SCExtShip".lower()] = SCExtShip
All_dfs["SRIntShip".lower()] = SRIntShip
All_dfs["SSIntShip".lower()] = SSIntShip

In [ ]:
filter_list = All_dfs["rcextship"]["ExtShip_id"]3


In [ ]:
filter_list = All_dfs["rcextship"]["ExtShip_id"]
All_dfs["rcextorders"] = All_dfs["externalorders"].query("ExtShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["scextship"]["ExtShip_id"]
All_dfs["scextorders"] = All_dfs["externalorders"].query("ExtShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["srintship"]["IntShip_id"]
All_dfs["srintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["ssintship"]["IntShip_id"]
All_dfs["ssintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)
All_dfs.keys()

dict_keys(['customer', 'externalorders', 'externalservices', 'externalshipments', 'externaltransactions', 'facilities', 'internalorders', 'internalservices', 'internalshipments', 'internaltransactions', 'manfacturing', 'products', 'retailer', 'supplier', 'warehouses', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

In [ ]:
All_dfs.pop("internalshipments")
All_dfs.pop("externalshipments")
All_dfs.pop("externalorders")
All_dfs.pop("internalorders")
All_dfs.keys()


dict_keys(['customer', 'externalservices', 'externaltransactions', 'facilities', 'internalservices', 'internaltransactions', 'manfacturing', 'products', 'retailer', 'supplier', 'warehouses', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

In [ ]:
All_dfs = {key: value for key, value in All_dfs.items() 
           if key in ["customer","supplier","retailer","rcextship","srintship","srintorders","rcextorders"]}
#Conflict

In [ ]:
All_dfs.keys()

dict_keys(['customer', 'externalservices', 'externaltransactions', 'facilities', 'internalservices', 'internaltransactions', 'manfacturing', 'products', 'retailer', 'supplier', 'warehouses', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

In [ ]:
#ref_in = {supplier :(fac,manf)}
pk = {}
fk = {}
ref_in = {}
table_name = list(All_dfs.keys())
for table_name in All_dfs:
    fk[table_name] = {}
    ref_in[table_name] = set()

In [ ]:
def pk_finder(name, df):
    for i in range(len(df.columns)):
        col = df.iloc[:,i]
        #if(col.size == col.drop_duplicates().size):
        if(col.nunique() == col.size):
            pk[name] = df.columns[i] 
            break

In [ ]:
def fk_finder(name, df):
    for i in range(len(df.columns)):
        col_name = df.columns[i]
        col = df.iloc[:,i]
        if(list(df.columns)[i] not in pk):
            flag = True
            for j in pk: #loops on the key j = table names
                if j != name:
                    primary_key_column = All_dfs[j][(pk[j])]
                    if primary_key_column.dtype == col.dtype:
                        status = col.isin(primary_key_column)
                        if status[status== False].size == 0:
                            fk[name][col_name] = j 
                            ref_in[j].add(name)
                            break

In [ ]:
for t in All_dfs:
    pk_finder(t,All_dfs[t])

In [ ]:
for t in All_dfs:
    fk_finder(t,All_dfs[t])

In [ ]:
#edges
#scextship: ['supplier','customer']
#nodes is a set
edges = {}
nodes = {}
properties = {}
ref_in

{'customer': {'externaltransactions', 'rcextship', 'scextship'},
 'externalservices': set(),
 'externaltransactions': {'externalservices'},
 'facilities': set(),
 'internalservices': set(),
 'internaltransactions': set(),
 'manfacturing': set(),
 'products': set(),
 'retailer': {'rcextship', 'srintship'},
 'supplier': {'externaltransactions',
  'facilities',
  'internaltransactions',
  'manfacturing',
  'scextship',
  'srintship',
  'ssintship'},
 'warehouses': set(),
 'rcextship': {'rcextorders'},
 'scextship': {'scextorders'},
 'srintship': {'srintorders'},
 'ssintship': {'ssintorders'},
 'rcextorders': set(),
 'scextorders': set(),
 'srintorders': set(),
 'ssintorders': set()}

In [ ]:
def nodes_edges(fk):
    for f in fk:
        if len(fk[f]) == 2:
            inner_dict = list(fk[f])
            edges[f] = fk[f]
        elif len(fk[f]) == 1:
            properties[f] = fk[f]
        else:
            nodes[f] = All_dfs[f]
    # for f in fk:
    #     if((f not in edges) ):
    #         nodes[f] = All_dfs[f]
    #         #nodes.append(f)
        

In [ ]:
ref_in

{'customer': {'externaltransactions', 'rcextship', 'scextship'},
 'externalservices': set(),
 'externaltransactions': {'externalservices'},
 'facilities': set(),
 'internalservices': set(),
 'internaltransactions': set(),
 'manfacturing': set(),
 'products': set(),
 'retailer': {'rcextship', 'srintship'},
 'supplier': {'externaltransactions',
  'facilities',
  'internaltransactions',
  'manfacturing',
  'scextship',
  'srintship',
  'ssintship'},
 'warehouses': set(),
 'rcextship': {'rcextorders'},
 'scextship': {'scextorders'},
 'srintship': {'srintorders'},
 'ssintship': {'ssintorders'},
 'rcextorders': set(),
 'scextorders': set(),
 'srintorders': set(),
 'ssintorders': set()}

In [ ]:
nodes_edges(fk)
edges 

{'rcextship': {'factoryIds/retailerIds': 'retailer', 'idsTo': 'customer'},
 'srintship': {'listSuppIds': 'supplier', 'factoryIds': 'retailer'}}

In [ ]:
nodes.keys()

dict_keys(['customer', 'retailer', 'supplier'])

In [ ]:
properties.keys()

dict_keys(['rcextorders', 'srintorders'])

In [ ]:
nodesTable = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
nodesTable


Empty DataFrame
Columns: [Label, ID, Attributes]
Index: []

In [ ]:
def add_nodes(nodes, nodesTable):
    for n in range(len(nodes)):
        for index, row in nodes[list(nodes.keys())[n]].iterrows():
            column_names = list(nodes[list(nodes.keys())[n]].columns) #get column names
            att = {}
            for i in range(1, len(column_names)):
                att[column_names[i]] = nodes[list(nodes.keys())[n]].iloc[index, i]
            newRow = [{'Label': list(nodes.keys())[n] , 'ID': nodes[list(nodes.keys())[n]].iloc[index,0] , 'Attributes': att}]
            tmp = pd.DataFrame(newRow)
            nodesTable = pd.concat([nodesTable, tmp], ignore_index=True)
    return nodesTable

        

In [ ]:
nodesTable = add_nodes(nodes,nodesTable)
nodesTable

Label     ID                                         Attributes
0    customer  33736  {'gender': 'F', 'first_name': 'Nadia', 'last_n...
1    customer  42959  {'gender': 'M', 'first_name': 'Eric', 'last_na...
2    customer  50653  {'gender': 'F', 'first_name': 'Alessia', 'last...
3    customer  82485  {'gender': 'M', 'first_name': 'Alexei', 'last_...
4    customer  56609  {'gender': 'F', 'first_name': 'Brenda', 'last_...
..        ...    ...                                                ...
595  supplier  90256  {'supplier_name': 'Förster', 'country': 'Ethio...
596  supplier  35266  {'supplier_name': 'Gotthard Zahn GmbH', 'count...
597  supplier  52647  {'supplier_name': 'Cook-Webb', 'country': 'Mor...
598  supplier  13355  {'supplier_name': 'Shah-Lowe', 'country': 'Cen...
599  supplier  48467  {'supplier_name': 'Brandt Ltd', 'country': 'An...

[600 rows x 3 columns]

In [ ]:
n = {"supplier": All_dfs["supplier"].head(5),"customer":All_dfs["customer"].head(5)}
df = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
df = add_nodes(n, df)
df

Label     ID                                         Attributes
0  supplier  34258  {'supplier_name': 'Taylor PLC', 'country': 'La...
1  supplier  40868  {'supplier_name': 'Sorrentino SPA', 'country':...
2  supplier  27813  {'supplier_name': 'Persico e figli', 'country'...
3  supplier  57551  {'supplier_name': 'Karge OHG mbH', 'country': ...
4  supplier  50099  {'supplier_name': 'Smith, Porter and Lynch', '...
5  customer  33736  {'gender': 'F', 'first_name': 'Nadia', 'last_n...
6  customer  42959  {'gender': 'M', 'first_name': 'Eric', 'last_na...
7  customer  50653  {'gender': 'F', 'first_name': 'Alessia', 'last...
8  customer  82485  {'gender': 'M', 'first_name': 'Alexei', 'last_...
9  customer  56609  {'gender': 'F', 'first_name': 'Brenda', 'last_...

In [ ]:
edgesTable = pd.DataFrame(columns = ['From_Node_ID', 'To_Node_ID', 'order/service'])
edgesTable

Empty DataFrame
Columns: [From_Node_ID, To_Node_ID, order/service]
Index: []

In [ ]:
edges

{'rcextship': {'factoryIds/retailerIds': 'retailer', 'idsTo': 'customer'},
 'srintship': {'listSuppIds': 'supplier', 'factoryIds': 'retailer'}}

In [ ]:
#order = {orderID: {}}
def add_edges(edges,edgesTable,pk,fk,ref_in):
    for e in edges:
        df = All_dfs[e] #df of the cur edge
        for r in range(len(df)):
            label = list(edges[e].keys())
            from_id = nodesTable[(nodesTable["Label"] == edges[e][label[0]]) & (df.loc[r,label[0]] == nodesTable["ID"])]
            from_id = from_id.index[0]
            to_id = nodesTable[(nodesTable["Label"] == edges[e][label[1]]) & (df.loc[r,label[1]] == nodesTable["ID"])]
            to_id = to_id.index[0]
            pk_col = pk[e]
#             primary_key = df[pk_col].iloc[r]
            primary_key = df.loc[r,pk_col]
            att = convert_prop(ref_in,e,primary_key,fk)
            newRow = [{'From_Node_ID': from_id, 'To_Node_ID': to_id, 'order/service' : att}]
            tmp = pd.DataFrame(newRow)
            edgesTable = pd.concat([edgesTable, tmp], ignore_index=True)
    return edgesTable


In [ ]:
#convert_prob(properties, properties,25462)
def convert_prop(ref_in,edge_key, pk_value,fk):
    for referenced_table_name in ref_in[edge_key]:
        referenced_table = All_dfs[referenced_table_name]
        fk_of_ref_table = fk[referenced_table_name]
        for foreign_key in fk_of_ref_table:
            if fk_of_ref_table[foreign_key] == edge_key:
#                 referenced_table[referenced_table[foreign_key]== id]
                all_occurances_df = referenced_table[referenced_table[foreign_key]== pk_value].drop([foreign_key],axis=1)
                return all_occurances_df.to_dict('records')
                

In [ ]:
edgesTable = add_edges(edges, edgesTable,pk,fk,ref_in)
edgesTable

From_Node_ID To_Node_ID                                      order/service
0            281        142  [{'ExtOrders_id': 3779, 'quantity': 961, 'plac...
1            216         84  [{'ExtOrders_id': 3158, 'quantity': 978, 'plac...
2            188         27  [{'ExtOrders_id': 9113, 'quantity': 205, 'plac...
3            252          5                                                 []
4            185         41  [{'ExtOrders_id': 1414, 'quantity': 436, 'plac...
..           ...        ...                                                ...
97           593        201  [{'IntOrders_id': 6274, 'quantity': 734, 'plac...
98           527        179  [{'IntOrders_id': 2364, 'quantity': 180, 'plac...
99           593        219  [{'IntOrders_id': 6461, 'quantity': 297, 'plac...
100          575        188                                                 []
101          352        251                                                 []

[102 rows x 3 columns]

In [ ]:

x_encoding =  {"supplier":50,"retailer":600,"customer":1000}
color_encoding = {"supplier":"#EEFFCC","retailer":"#E16A5B","customer":"#00A15F"}

In [ ]:
import numpy as np

list2 = [(x[1]+'_'+str(x[0])).capitalize() for x in zip(nodesTable.ID,nodesTable.Label)]
list2


['Customer_33736',
 'Customer_42959',
 'Customer_50653',
 'Customer_82485',
 'Customer_56609',
 'Customer_45986',
 'Customer_58600',
 'Customer_55976',
 'Customer_32433',
 'Customer_14142',
 'Customer_49539',
 'Customer_31548',
 'Customer_22043',
 'Customer_44945',
 'Customer_14574',
 'Customer_36708',
 'Customer_25357',
 'Customer_48071',
 'Customer_35953',
 'Customer_42770',
 'Customer_30824',
 'Customer_33597',
 'Customer_22036',
 'Customer_55905',
 'Customer_53365',
 'Customer_70649',
 'Customer_24642',
 'Customer_16956',
 'Customer_63453',
 'Customer_21156',
 'Customer_38534',
 'Customer_40624',
 'Customer_19561',
 'Customer_88373',
 'Customer_53519',
 'Customer_41538',
 'Customer_81331',
 'Customer_97225',
 'Customer_28053',
 'Customer_18779',
 'Customer_96892',
 'Customer_64367',
 'Customer_66152',
 'Customer_95167',
 'Customer_25491',
 'Customer_72090',
 'Customer_69145',
 'Customer_15307',
 'Customer_17040',
 'Customer_39892',
 'Customer_11346',
 'Customer_78650',
 'Customer_8

In [ ]:
nodes_color = [color_encoding[x] for x in list(nodesTable.Label)]
nodes_x_postion = [x_encoding[x] for x in list(nodesTable.Label)]

In [ ]:
indices = list(map(lambda x: int(x),list(nodesTable.index)))
# indices = list(range(len((list2))))

In [ ]:

nodes_attributes = list(map(lambda x: str(x).replace(',',"\n"),list(nodesTable.Attributes)))


In [ ]:

node_size = [300 for _ in range(len(nodesTable))]

In [ ]:
from pyvis.network import Network

g = Network(height='750px', width='100%')
g.barnes_hut()
g.add_nodes(indices,
            value = node_size,
            title= nodes_attributes,
            label=list2,
            color=nodes_color,
            group = 
            x=nodes_x_postion)
for e in range(len(edgesTable)):
    src = int(edgesTable.iloc[e]["From_Node_ID"])
    dst = int(edgesTable.iloc[e]["To_Node_ID"])
    g.add_edge(src,dst,weight=5)
    


g.show('test.html')

In [ ]:
import numpy as np
np.array(nodesTable.values.tolist())[:,0]

In [ ]:
for e in edgesTable.values:
    print(e)

[281 142
 list([{'ExtOrders_id': 3779, 'quantity': 961, 'placed_when': '1976-06-29', 'actual_date': nan, 'expected_date': '1976-07-18', 'cost': 1.72, 'status': 'Out For Delivery'}])]
[216 84
 list([{'ExtOrders_id': 3158, 'quantity': 978, 'placed_when': '2001-01-08', 'actual_date': '2001-01-18 00:00:00', 'expected_date': '2001-02-01', 'cost': 9.31, 'status': 'Closed'}, {'ExtOrders_id': 5348, 'quantity': 248, 'placed_when': '2002-08-30', 'actual_date': nan, 'expected_date': '2002-09-28', 'cost': 980.43, 'status': 'Out For Delivery'}])]
[188 27
 list([{'ExtOrders_id': 9113, 'quantity': 205, 'placed_when': '1978-01-30', 'actual_date': '1978-02-25 00:00:00', 'expected_date': '1978-02-28', 'cost': 825.35, 'status': 'Closed'}])]
[252 5 list([])]
[185 41
 list([{'ExtOrders_id': 1414, 'quantity': 436, 'placed_when': '1982-04-29', 'actual_date': nan, 'expected_date': '1982-05-23', 'cost': 4.93, 'status': 'Out For Delivery'}, {'ExtOrders_id': 7702, 'quantity': 172, 'placed_when': '1987-09-26', 'a

In [ ]:
nodesTable

Label     ID                                         Attributes
0    customer  33736  {'gender': 'F', 'first_name': 'Nadia', 'last_n...
1    customer  42959  {'gender': 'M', 'first_name': 'Eric', 'last_na...
2    customer  50653  {'gender': 'F', 'first_name': 'Alessia', 'last...
3    customer  82485  {'gender': 'M', 'first_name': 'Alexei', 'last_...
4    customer  56609  {'gender': 'F', 'first_name': 'Brenda', 'last_...
..        ...    ...                                                ...
595  supplier  90256  {'supplier_name': 'Förster', 'country': 'Ethio...
596  supplier  35266  {'supplier_name': 'Gotthard Zahn GmbH', 'count...
597  supplier  52647  {'supplier_name': 'Cook-Webb', 'country': 'Mor...
598  supplier  13355  {'supplier_name': 'Shah-Lowe', 'country': 'Cen...
599  supplier  48467  {'supplier_name': 'Brandt Ltd', 'country': 'An...

[600 rows x 3 columns]

In [ ]:
edgesTable

From_Node_ID To_Node_ID                                      order/service
0            281        142  [{'ExtOrders_id': 3779, 'quantity': 961, 'plac...
1            216         84  [{'ExtOrders_id': 3158, 'quantity': 978, 'plac...
2            188         27  [{'ExtOrders_id': 9113, 'quantity': 205, 'plac...
3            252          5                                                 []
4            185         41  [{'ExtOrders_id': 1414, 'quantity': 436, 'plac...
..           ...        ...                                                ...
97           593        201  [{'IntOrders_id': 6274, 'quantity': 734, 'plac...
98           527        179  [{'IntOrders_id': 2364, 'quantity': 180, 'plac...
99           593        219  [{'IntOrders_id': 6461, 'quantity': 297, 'plac...
100          575        188                                                 []
101          352        251                                                 []

[102 rows x 3 columns]

In [ ]:
import networkx as nx
nx_graph = nx.Graph()
for i in range(len(nodesTable)):
    node = nodesTable.iloc[i]
    nx_graph.add_node(i,size = 50,label = f"{node['Label']}_{node['ID']}".capitalize(),group=node["Label"], color=color_encoding[node["Label"]],x = x_encoding[node["Label"]])

for edge in edgesTable.iterrows():
    nx_graph.add_edge(edge[1]["From_Node_ID"],edge[1]["To_Node_ID"],weight=50)
    

In [ ]:
nx_graph.number_of_edges()


102

In [ ]:
# pos=nx.spring_layout(nx_graph,scale=2) 
# nx.draw(nx_graph,pos)
nt = Network(width="100%",height="200%")
#pulates the nodes and edges data structures
nt.from_nx(nx_graph)
nt.toggle_physics(True)

nt.show('nx.html')